In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream
import glob
import os
import pickle

2025-01-20 21:12:34.133895: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737400354.172807     405 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737400354.184158     405 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-20 21:12:34.226524: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def load_midi_files(dataset_path):
    """Load and process MIDI files from the given path."""
    notes = []
    
    # Get all MIDI files
    midi_files = glob.glob(os.path.join(dataset_path, "*.midi"))
    print(f"Processing {len(midi_files)} MIDI files...")
    
    # Process each file
    for i, file in enumerate(midi_files):
        try:
            print(f"Processing file {i+1}/{len(midi_files)}: {os.path.basename(file)}")
            midi = converter.parse(file)
            notes_to_parse = None
            
            try:
                # Try to get the piano part
                s2 = instrument.partitionByInstrument(midi)
                if s2:  # If there are instrument parts
                    notes_to_parse = s2.parts[0].recurse()
                else:  # If there are no instrument parts
                    notes_to_parse = midi.flat.notes
            except Exception:
                # If partitioning fails, use flat representation
                notes_to_parse = midi.flat.notes
            
            # Extract notes and chords
            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))
                    
        except Exception as e:
            print(f"Error processing {file}: {str(e)}")
            continue
            
    print(f"Successfully extracted {len(notes)} notes and chords")
    return notes


In [3]:
# Step 2: Prepare the input sequences and output labels
def prepare_sequences(notes, n_vocab, sequence_length=100):
    pitchnames = sorted(set(notes))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    network_input = network_input / float(n_vocab)  # Normalize input
    network_output = to_categorical(network_output)  # One-hot encode output

    return network_input, network_output, note_to_int

In [4]:
def build_model(input_shape, n_vocab):
    model = Sequential([
        LSTM(64, input_shape=input_shape, return_sequences=True),
        Dropout(0.2),
        LSTM(64),
        Dense(n_vocab, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model


In [5]:
# Step 4: Train the model
def train_model(model, network_input, network_output, epochs=10, batch_size=32):
    model.fit(network_input, network_output, epochs=epochs, batch_size=batch_size,validation_split=0.1,verbose=1)
    return model

In [6]:
def generate_music(model, network_input, pitchnames, n_vocab, sequence_length=50, num_notes=200):
    """
    Generate music with correct array shape handling
    """
    int_to_note = dict(enumerate(pitchnames))
    
    # Get random starting seed
    start = np.random.randint(0, len(network_input) - 1)
    pattern = network_input[start].flatten()  # Flatten to 1D array
    prediction_output = []

    # Pre-allocate normalized input array
    prediction_input = np.zeros((1, sequence_length, 1))
    scale_factor = float(n_vocab)

    for _ in range(num_notes):
        # Reshape pattern correctly
        prediction_input[0, :, 0] = pattern.flatten() / scale_factor

        # Get prediction
        index = np.argmax(model.predict(prediction_input, verbose=0))
        prediction_output.append(int_to_note[index])

        # Update pattern
        pattern = np.roll(pattern, -1)
        pattern[-1] = index

    return prediction_output

In [7]:
from music21 import instrument, note, chord, stream, tempo

def create_midi(prediction_output, filename="output.mid", instrument_type=instrument.Piano, min_duration_secs=10):
    """
    Create a MIDI file with a minimum duration of specified seconds
    Args:
        prediction_output: List of predicted notes/chords
        filename: Output MIDI filename
        instrument_type: MIDI instrument to use
        min_duration_secs: Minimum duration in seconds
    """
    # Create instrument instance once
    piano = instrument_type()
    output_notes = []
    
    # Calculate appropriate note duration
    total_notes = len(prediction_output)
    base_duration = min_duration_secs / (total_notes * 0.5)
    note_duration = max(base_duration, 0.25)
    
    # Add tempo marking for consistent playback
    tempo_mark = tempo.MetronomeMark(number=120)  # 120 BPM
    output_notes.append(tempo_mark)
    
    current_time = 0.0
    
    for i, pattern in enumerate(prediction_output):
        # Handle chords
        if ('.' in pattern):
            chord_notes = [
                note.Note(
                    int(current_note),
                    quarterLength=note_duration,
                    storedInstrument=piano
                )
                for current_note in pattern.split('.')
                if current_note.isdigit()
            ]
            if chord_notes:
                new_chord = chord.Chord(chord_notes)
                new_chord.offset = current_time
                output_notes.append(new_chord)
        
        # Handle single notes
        elif not isinstance(pattern, str) or pattern.isdigit():
            new_note = note.Note(
                int(pattern) if isinstance(pattern, str) else pattern,
                quarterLength=note_duration,
                storedInstrument=piano
            )
            new_note.offset = current_time
            output_notes.append(new_note)
        
        # Update timing
        current_time += note_duration
    
    # Create and write stream
    midi_stream = stream.Stream(output_notes)
    
    # Calculate actual duration
    actual_duration = current_time * 0.5  # Convert to seconds
    print(f"Created piece with duration: {actual_duration:.2f} seconds")
    
    midi_stream.write('midi', fp=filename)
    return actual_duration

In [8]:
# Main function
def main():
    # Load MIDI files
    notes = load_midi_files("maestro-v3.0.0-midi/maestro-v3.0.0/Made up")  # Replace with your MIDI file directory
    n_vocab = len(set(notes))

    # Prepare sequences
    sequence_length = 100
    network_input, network_output, note_to_int = prepare_sequences(notes, n_vocab, sequence_length)

    # Build and train the model
    model = build_model((network_input.shape[1], network_input.shape[2]), n_vocab)
    model = train_model(model, network_input, network_output, epochs=20, batch_size=128)

    # Generate music
    pitchnames = sorted(set(notes))
    prediction_output = generate_music(model, network_input, pitchnames, n_vocab, sequence_length, num_notes=500)

    # Save the generated music as a MIDI file
    create_midi(prediction_output, "generated_music2.mid")
    print("Music generated and saved as 'generated_music2.mid'")

if __name__ == "__main__":
    main()

Processing 13 MIDI files...
Processing file 1/13: MIDI-Unprocessed_01_R1_2008_01-04_ORIG_MID--AUDIO_01_R1_2008_wav--2.midi
Processing file 2/13: MIDI-Unprocessed_XP_15_R2_2004_01_ORIG_MID--AUDIO_15_R2_2004_03_Track03_wav.midi
Processing file 3/13: MIDI-Unprocessed_01_R1_2008_01-04_ORIG_MID--AUDIO_01_R1_2008_wav--1.midi
Processing file 4/13: MIDI-Unprocessed_01_R1_2009_01-04_ORIG_MID--AUDIO_01_R1_2009_01_R1_2009_02_WAV.midi
Processing file 5/13: MIDI-Unprocessed_R1_D1-9-12_mid--AUDIO-from_mp3_12_R1_2015_wav--1.midi
Processing file 6/13: MIDI-Unprocessed_Schubert10-12_MID--AUDIO_18_R2_2018_wav.midi
Processing file 7/13: MIDI-UNPROCESSED_09-10_R1_2014_MID--AUDIO_09_R1_2014_wav--3.midi
Processing file 8/13: MIDI-Unprocessed_01_R1_2009_01-04_ORIG_MID--AUDIO_01_R1_2009_01_R1_2009_01_WAV.midi
Processing file 9/13: MIDI-Unprocessed_Schubert10-12_MID--AUDIO_20_R2_2018_wav.midi
Processing file 10/13: MIDI-UNPROCESSED_09-10_R1_2014_MID--AUDIO_09_R1_2014_wav--2.midi
Processing file 11/13: MIDI-Unp

2025-01-20 21:19:42.654932: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/var/data/python/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20


2025-01-20 21:19:43.974024: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 165782904 exceeds 10% of free system memory.


324/324 ━━━━━━━━━━━━━━━━━━━━ 80s 227ms/step - loss: 5.8189 - val_loss: 5.7630
Epoch 2/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 73s 225ms/step - loss: 5.4450 - val_loss: 5.7660
Epoch 3/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 73s 224ms/step - loss: 5.4427 - val_loss: 5.7500
Epoch 4/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 73s 224ms/step - loss: 5.4178 - val_loss: 5.7335
Epoch 5/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 73s 226ms/step - loss: 5.3417 - val_loss: 5.7579
Epoch 6/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 73s 225ms/step - loss: 5.3043 - val_loss: 5.7798
Epoch 7/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 73s 225ms/step - loss: 5.2931 - val_loss: 5.7514
Epoch 8/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 72s 223ms/step - loss: 5.2635 - val_loss: 5.7937
Epoch 9/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 72s 223ms/step - loss: 5.2443 - val_loss: 5.7624
Epoch 10/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 72s 224ms/step - loss: 5.2260 - val_loss: 5.7854
Epoch 11/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 72s 223ms/step - loss: 5.2229 - val_loss: 5.8170
Epoch 12/20
324/324 ━━━━━━━━━━